# **1.올리브영 댓글 전처리**

In [ ]:
from konlpy.tag import Okt
import re
def preprocessing(text):
    with open('stop_words.txt','r') as f:
        stop_words = f.read().split(',')
    okt = Okt()
    rev_text=re.sub('[^가-힣\\s]','',text)
    word_review = okt.pos(rev_text,stem=True)
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token)>1]
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]
    return word_review

In [ ]:
import pandas as pd
import re
from tqdm.notebook import tqdm
from konlpy.tag import Okt
import json

# 전처리 함수
def preprocessing(text):
    with open('stop_words.txt','r') as f:
        stop_words = f.read().split(',')
    okt = Okt()
    rev_text=re.sub('[^가-힣\\s]','',text)
    word_review = okt.pos(rev_text,stem=True)
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token)>1]
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]
    return word_review

# 댓글 파일 open, 댓글 내용 없는 row drop
df_rev = pd.read_csv('goods_review_table_v2/reviews_A000000002839.csv',encoding='utf-8-sig')
df_rev.dropna(subset=['text'],inplace=True)


dic_word_rev={
    'text':[],
    'word':[],
    'score':[]
}
for i,item in df_rev.iterrows():
    word_rev = preprocessing(item['text'])
    dic_word_rev['text'].append(item['text'])
    dic_word_rev['word'].append(','.join(word_rev))
    dic_word_rev['score'].append(item['rev_score']//10)
x_train=dic_word_rev['word']
pd.DataFrame(dic_word_rev).head

<bound method NDFrame.head of                                                   text  \
0    정말 순하고 부드러운 토너예요\n한번도 트러블 일으킨 적 없고, 오히려 트러블로 범...   
1    좀 안맞는다는 생각이 들어서 자주사용하진 않지만 \n좀 건조해요 ,, 겨울엔 못 쓸...   
2    🍑내돈내산🍑\n바이오더마 센시비오 토너!\n전날 구입하고 하루 써보고 나서 같은 걸...   
3    🍑내돈내산🍑\n바이오더마 센시비오 토너 구입 후기입니당\n\n다들 많이 사용하고 있...   
4           민감성피부입니다. 자극 없이 순해서 편하게 사용할 수 있어요. 정착템입니다.   
..                                                 ...   
950  원래쓰던거라서 계속쓰는데 진짜 순하고 좋아요. 친구추천으로 첨썼는데 피뷰자극도 덜하...   
951  피부가 예민한 편이라 아무제품이나 쓰기가 무서웠는데 이 제품은 자극도 없고 순해서 ...   
952     피부를 진정시키는데 짱인것같아요.\n피부가 민감할때 스킨팩으로 쓰면 금방 진정됩니다   
953  세일기간에 좋은제품 싸게 잘 샀습니다. 클렌징 워터도 잘 쓰고있는데. 토너도 처음 ...   
954  피부의 좋은 변화는 못느꼈지만 뒤집어지지 않아서 그럭저럭 만족. 양이 적지만 닦토용...   

                                                  word  score  
0    정말,순하다,부드럽다,토너,한번,트러블,일으키다,적,없다,오히려,트러블,범벅,되다,...      1  
1           좀,맞다,생각,들어서다,자주,사용,하진,않다,좀,건조하다,겨울,못,쓸다,같다      1  
2    돈,산,바이오,덥다,세다,비다,토너,전날,구입,하루,써다,보고,나서다,같다,걸,재다...      1  
3    돈,산,바이오,덥다,세

# **2.올리브영 성분표 전처리**

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
from tqdm import tqdm

title = []
title_nonsep = []
product_id = []

# 제품명, 제품번호 크롤링

for i in tqdm(range(17)) :
  url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100080001&fltDispCatNo=&prdSort=01&pageIdx={}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000100080001_Small'.format(i+1)
  headers={
      'Accept':'*/*',
      'Accept-Encoding':'gzip, deflate, br',
      'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
      'Cennection':'keep-alive',
      'Host':'www.oliveyoung.co.kr',
      'sec-ch-ua':'".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
      'sec-ch-ua-mobile':'?0',
      'sec-ch-ua-platform':'"macOS"',
      'Sec-Fetch-Dest':'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'same-origin',
      'x-datadog-origin': 'rum',
      'x-datadog-parent-id': '7159823233067380424',
      'x-datadog-sampled': '1',
      'x-datadog-sampling-priority': '1',
      'x-datadog-trace-id': '1429624409174065303',
      'X-Requested-With': 'XMLHttpRequest',
      'Referer':'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000012880&dispCatNo=90000010001&trackingCd=Home_Curation1_1&curation=like&egcode=a016_a016&rccode=pc_main_01_c&egrankcode=2',
      'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
      'Cookie':'wl_chlNo=6; wl_chlDtlNo=11; RB_PCID=1658133194871774715; sch_check=yes; _gcl_au=1.1.2028396261.1658133196; PCID=16581331960867167840240; _gid=GA1.3.955099480.1658133196; oliveyoung_CID=d29d9b6d8ab0459feb86f0ff207b5f6e; oliveyoung_FIRSTDATE=1658133196468; _trs_id=eY07>7121555704152; EG_GUID=fb6f821a-4c07-40e2-b783-91c28135169b; _fbp=fb.2.1658133207261.257272098; _gcl_aw=GCL.1658133258.Cj0KCQjwidSWBhDdARIsAIoTVb1Sb4zQJXsucLe_A-Jaudb8aHkcBGthNa0rtrs6cVo-lfVwGY4JW2EaAs-SEALw_wcB; _gac_UA-92021806-1=1.1658133258.Cj0KCQjwidSWBhDdARIsAIoTVb1Sb4zQJXsucLe_A-Jaudb8aHkcBGthNa0rtrs6cVo-lfVwGY4JW2EaAs-SEALw_wcB; _gac_UA-181867310-1=1.1658133258.Cj0KCQjwidSWBhDdARIsAIoTVb1Sb4zQJXsucLe_A-Jaudb8aHkcBGthNa0rtrs6cVo-lfVwGY4JW2EaAs-SEALw_wcB; oliveyoung_LASTDATE=1658133258811; _gac_UA-92021806-9=1.1658133267.Cj0KCQjwidSWBhDdARIsAIoTVb1Sb4zQJXsucLe_A-Jaudb8aHkcBGthNa0rtrs6cVo-lfVwGY4JW2EaAs-SEALw_wcB; OYSESSIONID=16e665c4-da93-4479-9b71-82659ee1167b; NetFunnel_Main=; _trs_sid=G[6466435=5051[g[25<5303777526370; _trs_flow=; _gat_UA-92021806-9=1; _gat=1; _gat_UA-181867310-1=1; productHistory=[{"goodsNo":"A000000012880","viewCount":5},{"goodsNo":"A000000163275","viewCount":1},{"goodsNo":"A000000161313","viewCount":1},{"goodsNo":"A000000122076","viewCount":1},{"goodsNo":"A000000013651","viewCount":1},{"goodsNo":"A000000164615","viewCount":1}]; _ga_GMKKBJ29S2=GS1.1.1658207704.2.1.1658208594.29; wcs_bt=s_3ee47970f314:1658208594; RB_SSID=7uWsABAZaz; cto_bundle=CxdIFF8ybDZzTzJ4Vm1Ralp0emZydUJPbW82bVVuRlBGVSUyQiUyQnNNNmp6TGtXZUtFZHZ3V2ZXaVFpdUkxaHFqdDNQQ2tQJTJCTVVZeSUyRjVaT2lweTVOSTNZdUNwcUFHRnIwVEQzOVF2VnR0JTJCQzFkekJ3eFVWRVZ1em5sbjMwbHRqS1R4Mlh4T2tidEJobkhCRWQ5UVdhS0tCUEVpREklMkZ4JTJCUFFJeW5XUzVNcXJycVNoZm56YyUzRA; _dd_s=rum=0&expire=1658209497150; _ga_TTX3Z62VLN=GS1.1.1658207704.2.1.1658208597.0; _ga_P9V7281JHW=GS1.1.1658207714.2.1.1658208597.26; _ga=GA1.1.2092389701.1658133196'
  }

  resp = requests.get(url)
  soup = BeautifulSoup(resp.text, 'lxml')
  tit_tags = soup.select('p.tx_name')
  prodId_tags = soup.select('div.prd_name a.goodsList')
  
  for i,tit_tag in enumerate(tit_tags) :
    text1 = tit_tag.text
    
    if '기획' in tit_tag.text:
      continue
    elif '세트' in tit_tag.text and '+' in tit_tag.text :
      continue
    elif '세트' in tit_tag.text :
      continue
    elif '에디션' in tit_tag.text :
      continue
    elif '에멀전' in tit_tag.text :
      continue
    elif '로션' in tit_tag.text and not '토너' in tit_tag.text :
      continue
    elif '키트' in tit_tag.text :
      continue
    elif len(re.findall('[0-9]+종', text1)) > 0 :
      continue
    else:
      if ']' in text1:
        text1 = re.findall("] ?(.+)", text1)[0]
      if 'ml' in text1:
        text1 = re.findall("(.+?)[0-9]+ml", text1)[0]
      if 'ML' in text1 :
        text1 = re.findall('(.+?)[0-9]+ML', text1)[0]
      if 'mL' in text1 :
        text1 = re.findall('(.+?)[0-9]+mL', text1)[0]
      if '매' in text1 and len(re.findall('(.+?)\s[0-9]+매', text1))>0 :
        text1 = re.findall('(.+?)\s[0-9]+매', text1)[0]
      if 'oz' in text1 :
        text1 = re.findall('(.+?)[0-9].[0-9]+oz', text1)[0]
      if 'p' in text1 and len(re.findall('(.+?) [0-9]+p', text1)) > 0:
        text1 = re.findall('(.+?) [0-9]+p', text1)[0]  
      if '(' in text1 :
        text1 = re.findall("(.+?) ?\(", text1)[0]
      if text1[-1] == " ":
        text1 = str(text1[:-1])
      
      if text1.replace(" ", "") in title_nonsep:
        continue
      title.append(text1)
      title_nonsep.append(text1.replace(" ", ""))
      product_id.append(prodId_tags[i]['data-ref-goodsno'])


title.remove('아누아 어성초 77 토너')
product_id.remove('A000000148391')
title.remove('디오디너리 랙틱 애시드 5%+에이치에이')
product_id.remove('A000000141330')
title.remove('디오디너리 랙틱 애시드 10%+에이치에이')
product_id.remove('A000000141327')
title.remove('코스알엑스 퓨어핏 시카 토너')
product_id.remove('A000000137502')

dic_title = {
    'title': title,
    'product_id': product_id
}



df_title = pd.DataFrame(dic_title)
df_title

# df_title.to_csv('title_.csv', index = False, encoding = 'cp949')


100%|██████████| 17/17 [00:28<00:00,  1.67s/it]


,title,product_id
0,라운드랩 1025 독도 토너,A000000137180
1,에스트라 테라크네365 하이드레이션 토너,A000000126862
2,닥터지 레드 블레미쉬 클리어 수딩 토너,A000000154177
3,라로슈포제 에빠끌라 H 리밸런싱 하이드레이션 토너,A000000010474
4,아비브 어성초 카밍 토너 스킨부스터,A000000146589
...,...,...
243,CKD 레티노콜라겐 저분자 300 집중크림,A000000153784
244,CKD 레티노콜라겐 저분자 300 크림,A000000153778
245,아워 비건 어성초 시카 크림,A000000153727
246,아워 비건 어성초 시카 토너,A000000153724


In [ ]:
# #성분 크롤링
# pd.read_csv('ing_final1.csv', encoding = 'cp949')

comps = []
spliter = ['@', ',', '[]']
for i in tqdm(range(len(title))) : 
  # A = []

  url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsArtcAjax.do'

  data = {
      'goodsNo': product_id[i],
      'itemNo': '001',
      'pkgGoodsYn': 'N'
  }
  headers = {
      'referer' : 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={}&dispCatNo=100000100080008&trackingCd=Cat100000100080008_MID'.format(product_id[i]),
      'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.277 Whale/2.9.118.38 Safari/537.36',
      'cookie' : 'RB_PCID=1658133318588982757; sch_check=yes; _gcl_au=1.1.427087452.1658133321; PCID=16581333217657630606376; _gid=GA1.3.1793602515.1658133322; EG_GUID=7d6219c2-e188-47ef-91b0-f08d2a43adf3; _trs_id=eY07%3E7143453452%3E1; oliveyoung_CID=f481652d3d1c4887d5aeae7662fc53c8; OYSESSIONID=3fe5e44b-5d0a-48db-8830-642da67be94d; wl_chlNo=6; _trs_sid=G%5B6466435%3D5050%5Bg%5B25%3C5361671670%3C3; _trs_flow=; oliveyoung_FIRSTDATE=1658207656794; _fbp=fb.2.1658207737780.2010164342; wl_chlDtlNo=40; NetFunnel_Main=; oliveyoung_LASTDATE=1658208824851; RB_SSID=jxMP8rKOnZ; wcs_bt=s_3ee47970f314:1658209298; cto_bundle=fC2Sp19LV1h2WmZ4dm1wdXVQTzNmQU05RHd1SHJ0MGtQcGlXUm0lMkJsJTJCNUtnaU9GUng0blA4a0ozZkFWQVdHd3piamslMkJCM3ZhRXU2biUyRjdNeTY1ZDlsZTVqMkdYS0FJUm1WMU9ObEpSWmtKUkFpWVowRU05WDNkaSUyQklMY0d0NDI4Tm5EcXV4dDNGNDJuTU5qQUtWeEpYYVJFQXJ4VW84TlZFdCUyQnpqOTluWXdsdXklMkZZOVBQTkx0RFNQSmhMdDVsVTBUQ2I2S0tzOTklMkI3R052bTBlMzBwUThGUEVaQSUzRCUzRA; _ga=GA1.3.336530213.1658133322; _gat_UA-92021806-9=1; _ga_TTX3Z62VLN=GS1.1.1658207656.2.1.1658210598.0; _ga_P9V7281JHW=GS1.1.1658207663.2.1.1658210598.60; _ga_GMKKBJ29S2=GS1.1.1658207656.2.1.1658210598.60; productHistory=[{"goodsNo":"A000000126862","viewCount":6},{"goodsNo":"A000000165725","viewCount":1},{"goodsNo":"A000000169244","viewCount":1}]; _dd_s=rum=0&expire=1658211501771'
  }

  resp = requests.post(url,data = data, headers = headers)
  soup = BeautifulSoup(resp.text, 'lxml')
  comp_tags = soup.select('dl.detail_info_list dd')


  comp = str(comp_tags[6]).replace('<dd>','').replace('</dd>', '')#.replace('<br/>','')

  # if ',' not in comp :
  #   comp.replace(' ', ',')
  # if '[]' in comp :
  #   comp.replace('[]', ',')

  # if '(' in comp :
  #   comp = re.findall('([가-힣]+) ?(', comp)
  # if '*' in comp :
  #   comp = re.findall('([가-힣]+) ?*', comp)
  

  comps.append(comp.replace(' ', ''))

dic_components = {
    'title' : title,
    'components' : comps
}

df_comps = pd.DataFrame(dic_components)
# df_title.to_csv('title_.csv', index = False, encoding = 'cp949')
df_comps.to_csv('components__.csv', index = False, encoding='utf-8-sig')

100%|██████████| 248/248 [03:28<00:00,  1.19it/s]


In [ ]:
df_comps.loc[206].components = df_comps.loc[206].components.split('<br/>')[2]
df_comps.loc[67].components = df_comps.loc[67].components.split('<br/>')[1]
df_comps.loc[164].components = df_comps.loc[164].components.split('<br/>')[0]

In [ ]:
component = pd.read_csv('components__.csv')

In [ ]:
component

,title,components
0,라운드랩 1025 독도 토너,"정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수..."
1,에스트라 테라크네365 하이드레이션 토너,"정제수,부틸렌글라이콜,글리세린,녹차추출물,나이아신아마이드,프로판디올,페녹시에탄올,잔..."
2,닥터지 레드 블레미쉬 클리어 수딩 토너,"정제수,다이프로필렌글라이콜,프로판다이올,펜틸렌글라이콜,나이아신아마이드,부틸렌글라이콜..."
3,라로슈포제 에빠끌라 H 리밸런싱 하이드레이션 토너,정제수부틸렌글라이콜피이지-8글리세린소듐피씨에이피이지-60하이드로제네이티드캐스터오일살...
4,아비브 어성초 카밍 토너 스킨부스터,"정제수,약모밀추출물,글리세린,부틸렌글라이콜,펜틸렌글라이콜,자일리톨,비피다발효용해물,..."
...,...,...
243,CKD 레티노콜라겐 저분자 300 집중크림,"수용성콜라겐,글리세린,정제수,사이클로헥사실록세인,프로판다이올,나이아신아마이드,다이카..."
244,CKD 레티노콜라겐 저분자 300 크림,"수용성콜라겐,하이드로제네이티드폴리데센,카프릴릭/카프릭트라이글리세라이드,글리세린,펜타..."
245,아워 비건 어성초 시카 크림,"정제수,부틸렌글라이콜,약모밀추출물,카프릴릭/카프릭트라이글리세라이드,글리세린,다이카프..."
246,아워 비건 어성초 시카 토너,"정제수,약모밀추출물,부틸렌글라이콜,글리세레스-26,글리세린,병풀추출물,밤부사불가리스..."


In [ ]:
component['components']

0      정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수...
1      정제수,부틸렌글라이콜,글리세린,녹차추출물,나이아신아마이드,프로판디올,페녹시에탄올,잔...
2      정제수,다이프로필렌글라이콜,프로판다이올,펜틸렌글라이콜,나이아신아마이드,부틸렌글라이콜...
3      정제수부틸렌글라이콜피이지-8글리세린소듐피씨에이피이지-60하이드로제네이티드캐스터오일살...
4      정제수,약모밀추출물,글리세린,부틸렌글라이콜,펜틸렌글라이콜,자일리톨,비피다발효용해물,...
                             ...                        
243    수용성콜라겐(55%),글리세린,정제수,사이클로헥사실록세인,프로판다이올,나이아신아마이...
244    수용성콜라겐(53%),하이드로제네이티드폴리데센,카프릴릭/카프릭트라이글리세라이드,글리...
245    정제수,부틸렌글라이콜,약모밀추출물(어성초추출물,5%),카프릴릭/카프릭트라이글리세라이...
246    정제수,약모밀추출물(어성초추출물,5%),부틸렌글라이콜,글리세레스-26,글리세린,병풀...
247    정제수,변성알코올,프로판다이올,글리세레스-26,글리세린,나이아신아마이드,1,2-헥산...
Name: components, Length: 248, dtype: object

In [ ]:
for i in range(len(component['components'])):
  if '@' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('@', ',')
  elif '[]' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('[]', ',')
  elif ' ' in component['components'][i] :
    component['components'][i] = component['components'][i].replace(' ', ',')
  elif ',,' in component['components'][i] :
    component['components'][i] = component['components'][i].replace(',,', ',')
  #elif '<br/>' in component['components'][i] :
    #component['components'][i] = component['components'][i].replace('<br/>', '')
  #elif '1' in component['components'][i] :
    #component['components'][i] = component['components'][i].replace('1', '')
  elif '1,2-헥산다이올' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('1,2-헥산다이올', '2-헥산다이올')
  elif '1,2-헥산디올' in df_comps['components'][i] :
    component['components'][i] = component['components'][i].replace('1,2-헥산디올', '2-헥산다이올')
  elif ' ' in component['components'][i] :
    component['components'][i] = component['components'][i].replace(' ', '')
  elif ' ,' in component['components'][i] :
    component['components'][i] = component['components'][i].replace(' ,', '')    
  elif '2-헥산디올' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('2-헥산디올', '2-헥산다이올')
  elif '■,' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('■,', '')
  elif '■' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('■', '')
  elif '*' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('*', '')
  elif '+' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('+', '')
  elif "'" in component['components'][i] :
    component['components'][i] = component['components'][i].replace("'", '')
  elif '"""' in component['components'][i] :
    component['components'][i] = component['components'][i].replace('"""', '')

  component['components'][i] = re.sub('\([^)]*\)','',component['components'][i]) # ()안에 있는 문자열 포함 삭제
  component['components'][i] = re.sub('[0-9]*%','',component['components'][i]) #194번에 녹차추출물100% << 100%삭제
  component['components'][i] = re.sub('[0-9]*,[0-9]*ppm','',component['components'][i]) #32번에 750,000ppm 지우기
  component['components'][i] = re.sub('\[[가-힣]+\][1-9].+a','',component['components'][i])#242번 앞쪽 [본품],구성품50ml2ea 지우기
  component['components'][i] = re.sub('\[[가-힣]+\]','.',component['components'][i]) #[]안에 있는거 .으로 변환  55,83번 '&'으로 스플릿해서 가져옴
  #elif '*' in component['components'][i] :
    #component['components'][i] = re.findall('([가-힣]+)*', component['components'][i])
  #elif ",," in df_comps['components'][i]:
    #component['components'][i] = re.sub('(,+)',",",component['components'][i])
#\([^)]*\)/"""/\[[가-힣]+\][1-9].+a/\[[가-힣]+\]

component


,title,components
0,라운드랩 1025 독도 토너,"정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수..."
1,에스트라 테라크네365 하이드레이션 토너,"정제수,부틸렌글라이콜,글리세린,녹차추출물,나이아신아마이드,프로판디올,페녹시에탄올,잔..."
2,닥터지 레드 블레미쉬 클리어 수딩 토너,"정제수,다이프로필렌글라이콜,프로판다이올,펜틸렌글라이콜,나이아신아마이드,부틸렌글라이콜..."
3,라로슈포제 에빠끌라 H 리밸런싱 하이드레이션 토너,정제수부틸렌글라이콜피이지-8글리세린소듐피씨에이피이지-60하이드로제네이티드캐스터오일살...
4,아비브 어성초 카밍 토너 스킨부스터,"정제수,약모밀추출물,글리세린,부틸렌글라이콜,펜틸렌글라이콜,자일리톨,비피다발효용해물,..."
...,...,...
243,CKD 레티노콜라겐 저분자 300 집중크림,"수용성콜라겐,글리세린,정제수,사이클로헥사실록세인,프로판다이올,나이아신아마이드,다이카..."
244,CKD 레티노콜라겐 저분자 300 크림,"수용성콜라겐,하이드로제네이티드폴리데센,카프릴릭/카프릭트라이글리세라이드,글리세린,펜타..."
245,아워 비건 어성초 시카 크림,"정제수,부틸렌글라이콜,약모밀추출물,카프릴릭/카프릭트라이글리세라이드,글리세린,다이카프..."
246,아워 비건 어성초 시카 토너,"정제수,약모밀추출물,부틸렌글라이콜,글리세레스-26,글리세린,병풀추출물,밤부사불가리스..."


In [ ]:
#for tit in title:
  #component[tit][]
component[component['title'] =='라운드랩 1025 독도 토너'] 

,title,components
0,라운드랩 1025 독도 토너,"정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수..."


In [ ]:
component.loc[[0],['components']]


,components
0,"정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수..."


In [ ]:
component['components'][0]

'정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수추출물,해수,2-헥산다이올,프로테아제,베타인,판테놀,에틸헥실글리세린,알란토인,잔탄검,다이소듐이디티에이'

In [ ]:
df_comps['components'][6]

'정제수,다마스크장미꽃수,글리세린,에탄올,관중추출물,실새삼씨추출물,다마스크장미꽃오일,만난,알란토인,소듐하이알루로네이트,화이트윌로우잎추출물,올리브잎추출물,병풀추출물,베타-글루칸,감초추출물,쇠뜨기추출물,서양유채추출물,어성초추출물,밤껍질추출물,오배자추출물,황금추출물,모란뿌리추출물아이소이에서사용하는에탄올은식물발효성분입니다.'

In [ ]:
df_comps['components'][6] = re.sub('\([^)]*\)','',df_comps['components'][6])
df_comps['components'][6]

'정제수,다마스크장미꽃수,글리세린,에탄올,관중추출물,실새삼씨추출물,다마스크장미꽃오일,만난,알란토인,소듐하이알루로네이트,화이트윌로우잎추출물,올리브잎추출물,병풀추출물,베타-글루칸,감초추출물,쇠뜨기추출물,서양유채추출물,어성초추출물,밤껍질추출물,오배자추출물,황금추출물,모란뿌리추출물아이소이에서사용하는에탄올은식물발효성분입니다.'

In [ ]:
df_comps['components'][12]

'정제수,글리세린,마카다미아씨오일,스쿠알란,카프릴릭/카프릭트라이글리세라이드,펜타에리스리틸테트라아이소스테아레이트,세틸에틸헥사노에이트,트레할로오스,베타인,글리세릴스테아레이트,세테아릴알코올,알부틴,스테아릭애씨드,아라키딜알코올,메도우폼씨오일,솔비탄스테아레이트,다마스크장미꽃오일,시어버터,호호바씨오일,올리브오일,베헤닐알코올,아라키딜글루코사이드,다마스크장미꽃수,세라마이드엔피,아데노신,잔탄검,알란토인,팔미토일트라이펩타이드-5,참마뿌리추출물,밤껍질추출물,아세로라추출물,몰로키아잎추출물,브로콜리추출물,로즈마리추출물,니아울리나무잎오일,마트리카리아꽃/잎추출물,병풀추출물,락토바실러스/콩발효추출물,흰목이버섯추출물,황련뿌리추출물,부틸렌글라이콜,2-헥산다이올,스페인감초뿌리추출물,황금추출물,모란뿌리추출물,시트로넬올,제라니올,파네솔,유제놀,리날룰'

In [ ]:
df_comps['components'][12] = df_comps['components'][12].split('CeramidinComplex™')[0]

In [ ]:
df_comps['components'][31]

'병풀추출물,부틸렌글라이콜,2-헥산다이올'

In [ ]:
df_comps['components'][31] = df_comps['components'][31].split(':')[1]

In [ ]:
df_comps['components'][72]

'정제수,프로판다이올,글리세린,2-헥산다이올,만니톨,나이아신아마이드,알란토인,멘톡시프로판다이올,에틸헥실글리세린,카보머,다이소듐이디티에이,소듐하이알루로네이트,글루코노락톤,토코페롤'

In [ ]:
df_comps['components'][72] = df_comps['components'][72].split('※')[0]

In [ ]:
df_comps['components'][78]

'정제수,부틸렌글라이콜,글리세린,2-헥산다이올,흰버드나무껍질추출물,에틸헥실글리세린,피피지-13-데실테트라데세스-24,콘민트잎추출물,빌베리열매추출물,다이소듐이디티에이,사탕수수추출물,마데카소사이드,콘민트오일,글루코노락톤,오렌지추출물,레몬추출물,설탕단풍추출물,토코페롤,향료'

In [ ]:
df_comps['components'][78] = df_comps['components'][78].split(',SuperCica7™10,000PPM')[0]

In [ ]:
df_comps['components'][83]

'정제수,글리세린,프로판다이올,편백잎추출물*,2-헥산다이올,베타인,글리세레스-26,판테놀,에틸헥실글리세린,하이드롤라이즈드하이알루로닉애씨드,카보머,트로메타민,다이소듐이디티에이,알란토인,흰버드나무껍질추출물*,펜틸렌글라이콜,마데카소사이드,토코페롤'

In [ ]:
df_comps['components'][83] = df_comps['components'][83].split('.')[0]

In [ ]:
df_comps['components'][114]

'정제수,부틸렌글라이콜,에탄올,다이프로필렌글라이콜,코코넛야자수,1,2-헥산다이올,글리세린,피이지-60하이드로제네이티드캐스터오일,갈락토미세스발효여과물,베타인,소듐하이알루로네이트,향료,리모넨,페녹시에탄올,살리실릭애씨드,다이소듐이디티에이,에틸헥실글리세린'

In [ ]:
df_comps['components'][114] = df_comps['components'][114].split('Extract')[0]

In [ ]:
df_comps['components'][146]

'포트마리골드꽃추출물'

In [ ]:
df_comps['components'][146] = df_comps['components'][146].split('제공된')[0]

In [ ]:
df_comps['components'][147]

'정제수,다이프로필렌글라이콜,펜틸렌글라이콜,트라이에틸시트레이트,아이리쉬모스추출물,자일리톨,사탕수수추출물,아틀라스시다나무껍질오일,서양벌노랑이씨추출물,백미꽃추출물,파스향나무잎추출물,로즈마리잎오일,접시꽃추출물,클레리오일,캐모마일꽃오일,소듐하이알루로네이트,하이드롤라이즈드하이알루로닉애씨드,하이알루로닉애씨드,부틸렌글라이콜,판테놀,베타인,폴리글리세릴-2올리에이트,글리세린,다이페닐다이메티콘,트라이에틸헥사노인,에틸헥실글리세린,다이포타슘글리시리제이트,소듐시트레이트,석시닉애씨드,2-헥산다이올,하이드로제네이티드레시틴,시트릭애씨드,트로메타민,토코페롤,하이드록시프로필트라이모늄하이알루로네이트,소듐아세틸레이티드하이알루로네이트,하이드롤라이즈드소듐하이알루로네이트,소듐하이알루로네이트크로스폴리머,포타슘하이알루로네이트,카보머'

In [ ]:
df_comps['components'][147] = df_comps['components'][147].split('제공된')[0]

In [ ]:
df_comps['components'][170]

'알로에베라잎추출물,병풀추출물,글리세린,부틸렌글라이콜,나이아신아마이드,다이프로필렌글라이콜,베타인,트레할로오스,판테놀,바이오사카라이드검-1,시트릭애씨드,비피다발효여과물,매실추출물,연꽃추출물,하이드로제네이티드레시틴,마카다미아씨오일,살비아잎추출물,라벤더꽃추출물,로즈마리추출물,마트리카리아꽃추출물,카멜그라스추출물,세라마이드엔피,스쿠알란,세테아릴알코올,소듐하이알루로네이트,소듐하이알루로네이트크로스폴리머,하이드롤라이즈드하이알루로닉애씨드,하이알루로닉애씨드,하이드롤라이즈드소듐하이알루로네이트,망고추출물,얼룩파인애플추출물,락토바실러스발효물,폴리글리세릴-10올리에이트,아시아티코사이드,마데카식애씨드,아시아틱애씨드,하이드록시에틸아크릴레이트/소듐아크릴로일다이메틸타우레이트코폴리머,카보머,트로메타민,다이소듐이디티에이,하이드록시아세토페논,2-헥산다이올,에틸헥실글리세린'

In [ ]:
df_comps['components'][170] = df_comps['components'][170].split(':')[1]

IndexError: ignored

In [ ]:
df_comps['components'][184]

'정제수,베타인,글리세린,프로판다이올,흑미추출물,죽순껍질추출물,아스퍼질러스발효물,홍삼추출물,사이클로덱스트린,황금추출물,하이알루로닉애씨드,베타-글루칸,셀룰로오스검,잔탄검,부틸렌글라이콜,우스니아추출물,초피나무열매추출물,할미꽃추출물,소듐파이테이트,타마린드씨검,폴리글리세릴-10라우레이트,폴리글리세릴-10미리스테이트,글루코오스,2-헥산다이올,에탄올,라벤더오일'

In [ ]:
df_comps['components'][184] = df_comps['components'][184].replace('4,', '4')
df_comps['components'][184]

'정제수,베타인,글리세린,프로판다이올,흑미추출물,죽순껍질추출물,아스퍼질러스발효물,홍삼추출물,사이클로덱스트린,황금추출물,하이알루로닉애씨드,베타-글루칸,셀룰로오스검,잔탄검,부틸렌글라이콜,우스니아추출물,초피나무열매추출물,할미꽃추출물,소듐파이테이트,타마린드씨검,폴리글리세릴-10라우레이트,폴리글리세릴-10미리스테이트,글루코오스,2-헥산다이올,에탄올,라벤더오일'

In [ ]:
for i in range(len(title)):
  if '<br/>' in df_comps['components'][i] :
    df_comps['components'][i] = df_comps['components'][i].replace('<br/>', '')

In [ ]:
df_comps['components'][128] = df_comps['components'][128].split('<iln50331>')[0]

In [ ]:
df_comps['components'][217] = df_comps['components'][217].split('<iln48097>')[0]

In [ ]:
df_comps['components'][224] = df_comps['components'][224].split('<iln48589>')[0]

In [ ]:
df_comps

,components
title,
라운드랩 1025 독도 토너,"정제수,부틸렌글라이콜,글리세린,펜틸렌글라이콜,프로판다이올,아이리쉬모스추출물,사탕수수..."
에스트라 테라크네365 하이드레이션 토너,"정제수,부틸렌글라이콜,글리세린,녹차추출물,나이아신아마이드,프로판디올,페녹시에탄올,잔..."
닥터지 레드 블레미쉬 클리어 수딩 토너,"정제수,다이프로필렌글라이콜,프로판다이올,펜틸렌글라이콜,나이아신아마이드,부틸렌글라이콜..."
라로슈포제 에빠끌라 H 리밸런싱 하이드레이션 토너,정제수부틸렌글라이콜피이지-8글리세린소듐피씨에이피이지-60하이드로제네이티드캐스터오일살...
아비브 어성초 카밍 토너 스킨부스터,"정제수,약모밀추출물,글리세린,부틸렌글라이콜,펜틸렌글라이콜,자일리톨,비피다발효용해물,..."
...,...
CKD 레티노콜라겐 저분자 300 집중크림,"수용성콜라겐,글리세린,정제수,사이클로헥사실록세인,프로판다이올,나이아신아마이드,다이카..."
CKD 레티노콜라겐 저분자 300 크림,"수용성콜라겐,하이드로제네이티드폴리데센,카프릴릭/카프릭트라이글리세라이드,글리세린,펜타..."
아워 비건 어성초 시카 크림,"정제수,부틸렌글라이콜,약모밀추출물,카프릴릭/카프릭트라이글리세라이드,글리세린,다이카프..."


# **3.화장품 도메인 단어 토큰화 전처리 (Customized KoNLPy 라이브러리)**

In [ ]:
import pandas as pd

In [ ]:
!pip install customized_konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 881 kB 24.4 MB/s 
     |████████████████████████████████| 453 kB 40.2 MB/s 
     |████████████████████████████████| 19.4 MB 1.3 MB/s 


In [ ]:
import ckonlpy
from ckonlpy.tag import Twitter

In [ ]:
corpus_df = pd.read_csv('/content/drive/MyDrive/corpus_df.csv')

In [ ]:
ing = pd.read_csv('/content/drive/MyDrive/ing_final.csv')

In [ ]:
f = open('/content/drive/MyDrive/stop_words.txt')
stopwords = f.read().split(',')

In [ ]:
import re
twitter = Twitter()
twitter.add_dictionary([i for i in ing['표준 성분명']]+['흡수력', '끈적임', '트러블', '자극적', '내돈내산', '가성비', '사용감', '저자극', '민감성', '화농성', '블랙헤드', '대용량', '리무버', '밀착력', '유튜버', '강추', '수분감', '데일리', '유분기', '정착템', '바디로션', '클렌징오일', '피부결', '스킨팩', '제형', '속건조', '복합성', '지성', '건성', '클렌징워터', '재구매의사', '재구매', '재구매 의사', '닦토', '흡토', '수부지', '화장솜'], 'Noun')
twitter.add_dictionary(['가라앉다', '보들보들하다'], 'Adjective')

def preprocess(text):

  tokens = []

  non_kor_chars = re.compile('[^ 가-힣]')
  word = non_kor_chars.sub('', text)
  token = [t for (t,pos) in twitter.pos(text, stem=True) if pos in ['Noun', 'Verb', 'Adjective'] and t not in stopwords and len(t) > 1]
  preprocessed = ' '.join(token)
  return preprocessed

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
